In [1]:
import pandas as pd

# Load the CSV files
data_1920 = pd.read_csv('shots_laliga_1920.csv')
data_2021 = pd.read_csv('shots_laliga_2021.csv')
data_2122 = pd.read_csv('shots_laliga_2122.csv')

# Add a new column to each dataframe to denote the season year
data_1920['Year'] = '2019-2020'
data_2021['Year'] = '2020-2021'
data_2122['Year'] = '2021-2022'

# Combine all dataframes into one
combined_data_laliga = pd.concat([data_1920, data_2021, data_2122], ignore_index=True)

/var/folders/x1/tzn5_xnj79j6mrgx5h3grhw00000gn/T/ipykernel_75620/2505749828.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the CSV files
epl_data_1920 = pd.read_csv('shots_premier_league_1920.csv')
epl_data_2021 = pd.read_csv('shots_premier_league_2021.csv')
epl_data_2122 = pd.read_csv('shots_premier_league_2122.csv')

# Add a new column to each dataframe to denote the season year
epl_data_1920['Year'] = '2019-2020'
epl_data_2021['Year'] = '2020-2021'
epl_data_2122['Year'] = '2021-2022'

# Combine all dataframes into one
combined_data_england = pd.concat([epl_data_1920, epl_data_2021, epl_data_2122], ignore_index=True)


In [3]:
# Load the CSV files
bdl_data_1920 = pd.read_csv('shots_bundesliga_1920.csv')
bdl_data_2021 = pd.read_csv('shots_bundesliga_2021.csv')
bdl_data_2122 = pd.read_csv('shots_bundesliga_2122.csv')

# Add a new column to each dataframe to denote the season year
bdl_data_1920['Year'] = '2019-2020'
bdl_data_2021['Year'] = '2020-2021'
bdl_data_2122['Year'] = '2021-2022'

# Combine all dataframes into one
combined_data_germany = pd.concat([bdl_data_1920, bdl_data_2021, bdl_data_2122], ignore_index=True)


In [4]:
# Add a new column 'league' to each dataframe to denote the league origin
combined_data_germany['league'] = 'germany'
combined_data_england['league'] = 'england'
combined_data_laliga['league'] = 'spain'

# Combine all dataframes into one
combined_data = pd.concat([combined_data_germany, combined_data_england, combined_data_laliga], ignore_index=True)

In [6]:

import numpy as np
# Filter out rows where the 'Result' column contains 'own goal'
filtered_data = combined_data[combined_data['result'].str.lower() != 'own goal']

# Adjust the 'Result' column to have only 'Goal' or 'No Goal'
filtered_data['result'] = filtered_data['result'].apply(lambda x: 'Goal' if x.lower() == 'goal' else 'No Goal')

# Define the dimensions of the football field in meters
field_length = 105  # Length from goal to goal
field_width = 68    # Width across the field

# Calculate the shot distance
# Convert the X and Y from percentages to meters
# X represents the length and Y represents the width
# The center of the goal line is at (0%, 50%)
filtered_data['shot_distance'] = np.sqrt(
    ((filtered_data['X'] - 50) * field_width / 100) ** 2 + (filtered_data['Y'] * field_length / 100) ** 2
)

# Define the dimensions of the football field and goal in meters
field_width = 68    # Width across the field
goal_width = 7.32   # Width of the goal

# Define the function to calculate the angle
def calculate_visible_angle(x, y):
    # Convert percentages to meters
    x_meters = (x / 100) * field_width
    y_meters = (y / 100) * field_width
    
    # Distance from shot to each goal post
    distance_to_post_left = np.sqrt((x_meters - (field_width / 2 - goal_width / 2))**2 + y_meters**2)
    distance_to_post_right = np.sqrt((x_meters - (field_width / 2 + goal_width / 2))**2 + y_meters**2)
    
    # Angle to each post from the shot
    angle_left = np.arctan((field_width / 2 - goal_width / 2) / y_meters)
    angle_right = np.arctan((field_width / 2 + goal_width / 2) / y_meters)
    
    # Visible angle is the absolute difference between the two angles
    visible_angle = np.abs(angle_right - angle_left)
    
    # Convert radians to degrees for readability
    visible_angle_degrees = np.degrees(visible_angle)
    
    return visible_angle_degrees

# Apply the function to calculate the visible shot angle
filtered_data['angle_to_goal'] = filtered_data.apply(lambda row: calculate_visible_angle(row['X'], row['Y']), axis=1)

# Create a new column for the total goals by adding home and away goals
filtered_data['total_goals'] = filtered_data['h_goals'] + filtered_data['a_goals']

# Define a function to determine the half based on the minute
def determine_half(minute):
    if minute <= 45:
        return 1
    else:
        return 2

# Apply the function to create a new 'half' column
filtered_data['half'] = filtered_data['minute'].apply(determine_half)


# Drop the specified columns
filtered_data.drop(columns=['match_id', 'date', 'player_assisted'], inplace=True)

# Save the combined dataframe to a new CSV file
filtered_data.to_csv('final_project.csv', index=False)
